In [1]:
using Random, GMRF

include("src/dataGen.jl");
include("src/cavi.jl");
include("src/mcmc.jl");
include("src/plotting.jl");
include("src/evaluation.jl");

In [35]:
M₂ = 5;
M₁ = 5;
M = M₁ * M₂;

**WARNING**: save folder !

In [36]:
saveFolder = "results/$(M₁)x$(M₂)";
saveImgFolder = "results/$(M₁)x$(M₂)/images";

In [37]:
# saveFolder = "results/slow_convergence";
# saveImgFolder = "results/slow_convergence/images";

---

In [38]:
Random.seed!(300);
Fmu = iGMRF(M₁, M₂, 1, 1);
Fphi = iGMRF(M₁, M₂, 1, 10);
gridTarget = generateTargetGrid(Fmu, Fphi);
gridTarget[:, :, 1] = gridTarget[:, :, 1] .+ 40.0;
gridTarget[:, :, 2] = gridTarget[:, :, 2] .+ 1.0;
gridTarget[:, :, 3] = gridTarget[:, :, 3] .+ 0.05;
nobs = 100;
data = generateData(gridTarget, nobs);

In [39]:
nEpochMax = 12;
epochSize = 1;

initialValues = Dict(
    :μ => zeros(M),
    :ϕ => zeros(M),
    :ξ => 0.0,
    :kappaUparam => ((M - 1) / 2 + 1) / .1,
    :kappaVparam => ((M - 1) / 2 + 1) / 1,
);

spatialScheme = Dict(
    :M => M,
    :Fmu => Fmu,
    :Fphi => Fphi,
    :data => data,
);

In [ ]:
include("src/cavi.jl");

res = runCAVI(nEpochMax, epochSize, spatialScheme, initialValues=initialValues, saveFolder=saveFolder);

Itération 0...
Itération 1...
Inf
Itération 2...
22.24643712315221
Itération 3...
0.15750486838085634
Itération 4...
0.01056016104266746
Itération 5...
0.0004736551212408813
Itération 6...
0.0009175576750931213
Itération 7...
0.0028060974487380008
Itération 8...
0.0021185776178754166
Itération 9...
0.0011506779350020223
Itération 10...
0.003683864036938758
Itération 11...
0.0005349659670173424
Itération 12...
0.0014099849436752265
L'algorithme a atteint le nombre maximum d'itérations.


In [41]:
res.MCKL / 25

13-element Vector{Float64}:
  Inf
 285.54691826785273
 263.22249402272826
 263.06680997443755
 263.05334066471636
 263.05045156357215
 263.048858573613
 263.05402002068723
 263.0493476815125
 263.05009141656313
 263.05182440431656
 263.0498181238594
 263.04922530854293

In [34]:
plotConvergenceCriterion(res.MCKL, saveFolder=saveImgFolder);
plotTraceCAVI(res.traces[:muMean][200, :], "μ200", saveFolder=saveImgFolder);
plotTraceCAVI(res.traces[:phiMean][200, :], "ϕ200", saveFolder=saveImgFolder);
plotTraceCAVI(res.traces[:xiMean], "ξ", saveFolder=saveImgFolder);
trace = res.traces[:kappaUparams][1, :] ./ res.traces[:kappaUparams][2, :];
plotTraceCAVI(trace, "κᵤ", saveFolder=saveImgFolder);
trace = res.traces[:kappaVparams][1, :] ./ res.traces[:kappaVparams][2, :];
plotTraceCAVI(trace, "κᵥ", saveFolder=saveImgFolder);

In [38]:
res.MCKL/40000

6-element Vector{Float64}:
  Inf
 261.73274497111095
 261.7313067603047
 261.73137786255586
 261.73134961509277
 261.7314100286413

# MCMC

In [25]:
datastructure = Dict(
    :Y => data,
    :Fmu => Fmu,
    :Fphi => Fphi,
);

niter = 10000;

initialvalues = Dict(
    :μ => zeros(M),
    :ϕ => zeros(M),
    :ξ => .1,
    :κᵤ => .1,
    :κᵥ => .1,
);

stepsize = Dict(
    :μ => 1.3,
    :ϕ => .4,
    :ξ => .002,
);

In [26]:
include("src/mcmc.jl")

chain = mcmc(datastructure, niter, initialvalues, stepsize, saveFolder=saveFolder);

Progress:  46%|██████████████████▊                      |  ETA: 22:04:59KExcessive output truncated after 524353 bytes.

In [27]:
changerate(chain)

             Change Rate
          μ1       0.146
          μ2       0.153
          μ3       0.146
          μ4       0.144
          μ5       0.185
          μ6       0.169
          μ7       0.159
          μ8       0.142
          μ9       0.184
         μ10       0.152
         μ11       0.144
         μ12       0.167
         μ13       0.137
         μ14       0.146
         μ15       0.175
         μ16       0.124
         μ17       0.150
         μ18       0.136
         μ19       0.137
         μ20       0.207
         μ21       0.215
         μ22       0.184
         μ23       0.167
         μ24       0.185
         μ25       0.195
         μ26       0.179
         μ27       0.190
         μ28       0.257
         μ29       0.251
         μ30       0.218
         μ31       0.267
         μ32       0.209
         μ33       0.202
         μ34       0.165
         μ35       0.245
         μ36       0.268
         μ37       0.230
         μ38       0.184
         μ39       0.159


In [28]:
plotTraceMCMC(chain, "ξ", saveFolder=saveImgFolder);

In [35]:
include("src/plotting.jl");

plotCAVIvsMCMC(20000, caviRes=res, mcmcChain=chain, warmingSize=1000, saveFolder=saveImgFolder);

In [30]:
compDistance(res.traces, gridTarget)

54.23367576545461

In [31]:
compDistance(chain, gridTarget, 1000)

54.19370856067864